In [275]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

In [311]:
def gme_1(i, j):
    return np.sqrt(1/(i+1) + 1/(j+1))

def hilbert(i, j):
    return 1 / (i + j + 1)

def gme_2(i, j):
    return np.sqrt(i + j)

def gme_3(i, j):
    return np.sin(i + j)

def ones(i, j):
    return ((i+1) % (i+1) + ((j + 1) % (j + 1)) + 1)

In [335]:
def stop_criterion(uv_frobenius_norm, e, n, m, rank, ij_element):
    return e * uv_frobenius_norm >= np.abs(ij_element) * np.sqrt((m - rank) * (n - rank))


def get_row(get_element, n, m, i):
    return np.fromfunction(lambda j: get_element(i, j), (m,), dtype=float)


def get_col(get_element, n, m, j):
    return np.fromfunction(lambda i: get_element(i, j), (n,), dtype=float)


def get_row_UV(U, V, n, m, i):
    return np.reshape((U @ V)[i, :], (m,))


def get_col_UV(U, V, n, m, j):
    return np.reshape((U @ V)[:, j], (n,))


def find_max_element_in_matrix(get_element, n, m, start_column):
    matrix_row = get_col(get_element, n, m, start_column)
    row_index = np.argmax(np.abs(matrix_row))

    matrix_col = get_row(get_element, n, m, row_index)
    col_index = np.argmax(np.abs(matrix_col))
    return row_index, col_index


def find_max_element(get_element, n, m, U, V, start_column):
    matrix_col = get_col(get_element, n, m, start_column)
    uv_col     = get_col_UV(U, V, n, m, start_column)
    row_index = np.argmax(np.abs(matrix_col - uv_col))

    matrix_row = get_row(get_element, n, m, row_index)
    uv_row     = get_row_UV(U, V, n, m, row_index)
    row = matrix_row - uv_row
    col_index = np.argmax(np.abs(row))
    return row_index, col_index, row[col_index]


def norm_update(past_norm, U, V, u, v):
    return np.linalg.norm(U @ V)
#    return np.sqrt(past_norm**2 + 2 * np.real(np.dot(U.T @ u, V @ v)) + (np.linalg.norm(u) * np.linalg.norm(v))**2)


def adaptive_cross(get_element, n, m, e):
    I = [i for i in range(n)]
    J = [j for j in range(m)]
    I_z = []
    J_z = []
    r = 1

    # first iteration
    i, j = find_max_element_in_matrix(get_element, n, m, J[0])
    J_z.append(j)
    I_z.append(i)
    ij_el = get_element(i, j)
    U = np.reshape((get_col(get_element, n, m, j)) / np.abs(ij_el), (n, 1))
    V = np.reshape((get_row(get_element, n, m, i)) * np.abs(ij_el) / ij_el, (1, m))
    norm = U.T @ V.T
    print(norm)
    
    while not stop_criterion(norm, e, n, m, r, ij_el):
        r += 1
        # finding maxvol
        J_new = (list(set(J) - set(J_z)))
        i, j, ij_el = find_max_element(get_element, n, m, U, V, J_new[0])
        J_z.append(j)
        I_z.append(i)
        
        # getting respective row and column
        u = (get_col(get_element, n, m, j) - get_col_UV(U, V, n, m, j)) / np.abs(ij_el)
        v = (get_row(get_element, n, m, i) - get_row_UV(U, V, n, m, i)) * np.abs(ij_el) / ij_el
        # evaluate stopping criterion
        norm = norm_update(norm, U, V, u, v)
        print(norm)
        # updating U, V
        U = np.c_[U, u]
        V = np.r_['0,2', V, v]
        
    return U, V, I_z, J_z 

In [333]:
n = 4096
function = gme_2
matrix = np.fromfunction(function, (n, n), dtype=float)
print(matrix)

[[ 0.          1.          1.41421356 ... 63.97655821 63.98437309
  63.99218702]
 [ 1.          1.41421356  1.73205081 ... 63.98437309 63.99218702
  64.        ]
 [ 1.41421356  1.73205081  2.         ... 63.99218702 64.
  64.00781202]
 ...
 [63.97655821 63.98437309 63.99218702 ... 90.47651629 90.48204242
  90.48756821]
 [63.98437309 63.99218702 64.         ... 90.48204242 90.48756821
  90.49309366]
 [63.99218702 64.         64.00781202 ... 90.48756821 90.49309366
  90.49861877]]


In [329]:
rank = 10

In [ ]:
U, S, Vh = np.linalg.svd(matrix)
U = U[:, 0:rank]
Vh = Vh[0:rank, :]
S = S[0:rank]

In [ ]:
U1, V1h, I_z, J_z = adaptive_cross(function, n, n, 1e-3)
print(U1.shape)
print(V1h.shape)
print(np.linalg.norm(U1 @ V1h - matrix))

In [ ]:
# Determinant of find submatrix
I, J = np.meshgrid(I_z, J_z)
subm = function(I, J)
print(np.linalg.det(subm))